# Multidimensional data analysis

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

# Make the graphs a bit prettier, and bigger
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (15, 7)

In [ ]:
conn_string_fb = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
    user='student',
    password='dwdstudent2015',
    host='db.ipeirotis.org',
    port=3306,
    db='facebook')
engine_fb = create_engine(conn_string_fb)

## Multidimesnional data analysis: From raw counts to percentages

Now let's run a query to get the political views of Facebook users, broken down by gender.

In [ ]:
polviews_by_gender = '''
SELECT Sex, PoliticalViews, COUNT(*) AS cnt 
FROM Profiles 
WHERE Sex IS NOT NULL AND PoliticalViews IS NOT NULL 
GROUP BY Sex, PoliticalViews
ORDER BY  PoliticalViews, Sex
'''

And let's get the dataframe:

In [ ]:
df = pd.read_sql(polviews_by_gender, con=engine_fb)
df

In [ ]:
df.dtypes

In [ ]:
# Let's plot this!
# Bleh, this is ugly...
# Remember that the index of the dataframe becomes the default x-axis
df.plot(kind='bar')

In [ ]:
# Pivot, baby!
# Now the index contains the Political Views, which will be our x-axis
dfp = pd.pivot_table(
    data = df, 
    index='PoliticalViews', 
    columns='Sex', 
    values='cnt', 
    aggfunc='sum'
)
dfp

We will now convert the PoliticalViews column into an **ordered Categorical variable**. This is not strictly necessary, but it will be useful later.
 It ensures that Political Views appear in an order according to their political spectrum, as opposed to alphabetical.

In [ ]:
df.PoliticalViews = pd.Categorical(df.PoliticalViews,
    categories = ['Very Liberal', 'Liberal', 'Moderate', 'Conservative', 'Very Conservative', 'Libertarian', 'Apathetic', 'Other'], 
    ordered=True
)

dfp = pd.pivot_table(data = df, index='PoliticalViews', columns='Sex', values='cnt', aggfunc='sum')
dfp

In [ ]:
dfp.plot(kind='bar')

## Normalizing values: From raw counts to percentages

Now, let's see a bit how we can normalize the values in Pandas, by performing operations on the columns and rows.

In [ ]:
dfp

In [ ]:
# the .T is a shortcut for .transpose()
# and returns the transposed table
dfp.T

In [ ]:
# sums the columns
dfp.sum() 

# Equivalent commands:
# dfp.sum(axis='index')  # summing 'across the index'
# dfp.sum(axis=0)        # axis=0 is equivalent to summing axis='index'

In [ ]:
# Summing the transpose is same as 
# summing the rows of the original
dfp.T.sum()

# Equivalent commands
# dfp.sum(axis='columns') 
# dfp.sum(axis=1)

In [ ]:
# Let's normalize the columns, as we have more females than males, and it seems that there are always more women
dfp_norm = dfp / dfp.sum()
# Columns (ie Sex) sum up to 1
dfp_norm

In [ ]:
# Instead of dfp / dfp.sum(), we can also use the .div() method, for dividing the entries with the sum()
# Note that, by definition, the dfp / dfp.sum() operation divides  column-wise, not row-wise.
# dfp_norm = dfp.div( dfp.sum(), axis='columns' )
# dfp_norm

In [ ]:
dfp_norm.plot(kind='bar')

In [ ]:
# We will now normalize across the rows
dfp_norm2 = dfp.T / dfp.T.sum()
dfp_norm2.T

In [ ]:
# Same operation as above, but with the .div() function
# to find the fraction of males/females within each political category
# dfp_norm2 = dfp.div( dfp.sum(axis='columns'), axis='index' )
# dfp_norm2

In [ ]:
dfp_norm2.plot(kind='bar', rot=45)

In [ ]:
dfp_norm2.T.plot(kind='bar', stacked=True, rot=45)

## Exercise on Normalizing Values

a. Use the tables `RelationshipStatus` and `LookingFor`, and show create a plot with a breakdown of what people in different relationship statuses are looking for. To make things more readable (and to practice a bit SQL), remove from the output all combinations that have less than 10 students in them. The plot can use the absolute counts.
 


b. Normalize the results and plot again. To get experience with normalization, try to normalize both by Status (eg "80% of the people who are in a relationship are looking for Friendship") and by Relationship Status (eg "70% of the people who are looking for Random Play are Single").

### Solution (Part a)

In [ ]:
query = '''
SELECT R.Status, L.LookingFor, COUNT(*) AS cnt
FROM Relationship R INNER JOIN LookingFor L ON R.ProfileID = L.ProfileID
GROUP BY R.Status, L.LookingFor
HAVING cnt>10
'''
df = pd.read_sql(query, con=engine_fb)
df

In [ ]:
pivot = df.pivot_table(
    index='Status',
    columns='LookingFor',
    values='cnt'
)
pivot

In [ ]:
# Raw numbers, by relationship status
pivot.plot(kind='barh', figsize=(10,5))

In [ ]:
# Raw numbers, by looking for (taking the transpose)
pivot.T.plot(kind='barh', figsize=(10,5))

In [ ]:
# Every column (LookingFor) sums up to 1
normed1 = pivot / pivot.sum()
normed1

In [ ]:
# For every LookingFor value, show the breakdown among relationship statuses
normed1.T.plot.barh()

In [ ]:
# Every column (Status) sums up to 1
normed2 = (pivot.T / pivot.T.sum())
normed2

In [ ]:
# For every relationship status, show the breakdown among "LookingFor" values
normed2.T.plot.barh()